# <u> GIF displaying daily wind direction arrows for NUTS-3 regions in Germany </u>

The following code plots the daily average wind speed and directions for NUTS-3 regions in Germany. Wind speed and directions are displayed by arrows starting from the center point of each NUTS-3 region. Daily wind data has been extracted from the [ERA5-Land data](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview) and aggregated over NUTS-3 regions.
<br/> <br/>
Import required libararies.

In [1]:
import imageio
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

Read csv-file containing daily v- and u-component of wind for NUTS-3 regions in Germany into DataFrame.

In [2]:
# Path
PATH = "C:/Users/u0120816/OneDrive - KU Leuven/KU Leuven/Code_Example/"

# Load data
df_wind = pd.read_csv(PATH+"NUTS3_wind_2019.csv", sep = ';')

# Format datetime
df_wind['date']= pd.to_datetime(df_wind['date'])
df_wind

,NUTS_ID,date,eastward_wind_in_m_s,northward_wind_in_m_s,month,country
0,DE94E,2019-01-01,4.862784,-1.407169,1,DE
1,DE94E,2019-01-02,1.969943,-2.969080,1,DE
2,DE94E,2019-01-03,1.570592,-1.245010,1,DE
3,DE94E,2019-01-04,3.227463,-0.765708,1,DE
4,DE94E,2019-01-05,3.625280,-2.738789,1,DE
...,...,...,...,...,...,...
12023,DEG0N,2019-01-27,2.237816,3.519923,1,DE
12024,DEG0N,2019-01-28,3.339417,1.800007,1,DE
12025,DEG0N,2019-01-29,1.886753,1.094157,1,DE
12026,DEG0N,2019-01-30,-1.268157,2.058790,1,DE


Import NUTS-3 region shapefiles into GeoDataFrame using the [GISCO data distribution API](https://gisco-services.ec.europa.eu/distribution/v2/nuts/) and calculate the centroid of each NUTS-3 region.

In [3]:
url = "https://gisco-services.ec.europa.eu/distribution/v2/nuts/geojson/NUTS_RG_01M_2021_4326_LEVL_3.geojson"
gdf_nuts = gpd.read_file(url)
print("CRS of the Shapefile: " + str(gdf_nuts.crs))

# Restrict for NUTS-3 regions in Germany
gdf_nuts = gdf_nuts.loc[gdf_nuts.CNTR_CODE=="DE"]

# Reproject to Projected-CRS (EPSG:4326 is a Geographic-CRS inaccurate for determining centroids)
gdf_nuts = gdf_nuts.to_crs("EPSG:3395") 

# Calculate the centroid of each of each NUTS-3 region
nuts_center = gdf_nuts.copy()
nuts_center.crs =gdf_nuts.crs
nuts_center.geometry = nuts_center['geometry'].centroid
nuts_center

CRS of the Shapefile: epsg:4326


,id,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry
3,DE21A,DE21A,3,DE,Erding,Erding,4,3,3,DE21A,POINT (1335910.559 6125071.207)
4,DE94E,DE94E,3,DE,"Osnabrück, Landkreis","Osnabrück, Landkreis",4,2,3,DE94E,POINT (895534.532 6835738.572)
5,DE94F,DE94F,3,DE,Vechta,Vechta,4,2,3,DE94F,POINT (915400.631 6886457.109)
6,DE94G,DE94G,3,DE,Wesermarsch,Wesermarsch,4,2,1,DE94G,POINT (933995.190 7014884.689)
7,DEA59,DEA59,3,DE,Olpe,Olpe,2,2,3,DEA59,POINT (887943.166 6603299.704)
...,...,...,...,...,...,...,...,...,...,...,...
704,DE94H,DE94H,3,DE,Wittmund,Wittmund,4,3,1,DE94H,POINT (857786.535 7055866.972)
705,DEA11,DEA11,3,DE,"Düsseldorf, Kreisfreie Stadt","Düsseldorf, Kreisfreie Stadt",4,1,3,DEA11,POINT (758108.979 6629752.113)
804,DEG0L,DEG0L,3,DE,Greiz,Greiz,4,2,3,DEG0L,POINT (1344081.703 6543849.269)
805,DEG0M,DEG0M,3,DE,Altenburger Land,Altenburger Land,4,2,3,DEG0M,POINT (1380258.472 6580407.930)


Merge Wind-Dataframe with the NUTS-3-region-GeoDataFrame.

In [4]:
nuts_wind = nuts_center.merge(df_wind, on='NUTS_ID', how="left")
nuts_wind["x"] = nuts_wind.centroid.x
nuts_wind["y"] = nuts_wind.centroid.y
nuts_wind

,id,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry,date,eastward_wind_in_m_s,northward_wind_in_m_s,month,country,x,y
0,DE21A,DE21A,3,DE,Erding,Erding,4,3,3,DE21A,POINT (1335910.559 6125071.207),2019-01-01,4.232850,0.361723,1.0,DE,1.335911e+06,6.125071e+06
1,DE21A,DE21A,3,DE,Erding,Erding,4,3,3,DE21A,POINT (1335910.559 6125071.207),2019-01-02,4.156707,-2.948613,1.0,DE,1.335911e+06,6.125071e+06
2,DE21A,DE21A,3,DE,Erding,Erding,4,3,3,DE21A,POINT (1335910.559 6125071.207),2019-01-03,3.269911,-1.922807,1.0,DE,1.335911e+06,6.125071e+06
3,DE21A,DE21A,3,DE,Erding,Erding,4,3,3,DE21A,POINT (1335910.559 6125071.207),2019-01-04,3.483768,-0.947008,1.0,DE,1.335911e+06,6.125071e+06
4,DE21A,DE21A,3,DE,Erding,Erding,4,3,3,DE21A,POINT (1335910.559 6125071.207),2019-01-05,5.414622,0.219329,1.0,DE,1.335911e+06,6.125071e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12036,DEG0N,DEG0N,3,DE,"Eisenach, Kreisfreie Stadt","Eisenach, Kreisfreie Stadt",4,3,3,DEG0N,POINT (1146592.323 6586237.068),2019-01-27,2.237816,3.519923,1.0,DE,1.146592e+06,6.586237e+06
12037,DEG0N,DEG0N,3,DE,"Eisenach, Kreisfreie Stadt","Eisenach, Kreisfreie Stadt",4,3,3,DEG0N,POINT (1146592.323 6586237.068),2019-01-28,3.339417,1.800007,1.0,DE,1.146592e+06,6.586237e+06
12038,DEG0N,DEG0N,3,DE,"Eisenach, Kreisfreie Stadt","Eisenach, Kreisfreie Stadt",4,3,3,DEG0N,POINT (1146592.323 6586237.068),2019-01-29,1.886753,1.094157,1.0,DE,1.146592e+06,6.586237e+06
12039,DEG0N,DEG0N,3,DE,"Eisenach, Kreisfreie Stadt","Eisenach, Kreisfreie Stadt",4,3,3,DEG0N,POINT (1146592.323 6586237.068),2019-01-30,-1.268157,2.058790,1.0,DE,1.146592e+06,6.586237e+06


Loop over all days within a month, plot daily wind speed and directions for NUTS-3 regions, and store plots temporarly.

In [5]:
filenames = []
end = nuts_wind.date.dt.day.max()+1
for i in range(1, int(end)):

    try:

        # Select wind for a specific date
        nuts_wind_sel = nuts_wind.loc[nuts_wind.date == "2019-01-"+str(i).zfill(2)]

        # Plot NUTS-3 and neighboring regions
        fig, ax = plt.subplots(1,1, figsize=(20,20))
        csfont = {'fontname':'Times New Roman'}
        plt.tight_layout()
        plt.axis('off')
        plt.title("Wind on 2019-01-"+str(i).zfill(2), fontsize=40, **csfont)

        # Plot NUTS-3 rgeions
        gdf_nuts.plot(ax =ax, facecolor='grey', edgecolor="white", linewidth=0.5)

        # Plot arrows showing wind speed and direction
        plt.quiver(nuts_wind_sel["x"], nuts_wind_sel["y"], nuts_wind_sel['eastward_wind_in_m_s'], 
        nuts_wind_sel['northward_wind_in_m_s'], color="blue")

        # Legend
        arrow = mlines.Line2D([], [], color='blue', marker=r'$\rightarrow$', linestyle='none',
                                markersize=20, label='Wind Speed')
        plt.legend(handles=[arrow,], facecolor='none', labelcolor='black', prop={'size': 20, "family":"Times New Roman"}, loc="lower right")
       
        # Create file name and append it to a list
        filename = f'{i}.png'
        filenames.append(filename)

        # Save frame
        plt.savefig(filename, dpi=100, bbox_inches = 'tight')
        plt.close()

    except:
        continue

Build the GIF.

In [6]:
with imageio.get_writer(PATH+'Wind.gif', mode='I', fps=1.5) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in set(filenames):
    os.remove(filename)
print('GIF Done !')

GIF Done !


<img src="Wind.gif" width="750" align="center">